# Example Two: Animals-10

**In this example, I will do the following:**

    1. Build my own model and eval
    2. Use pretrained model and best weights possible and compare
    3. Choose the best model and plot the loss curves and confusion matrix

In [7]:
import torch
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import torchmetrics

from torch import nn
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
from torchmetrics import Accuracy, ConfusionMatrix
from torch.utils.data import DataLoader
from mlxtend.plotting import plot_confusion_matrix
from tqdm.auto import tqdm
from torch.utils.tensorboard import SummaryWriter

In [8]:
print(f'PyTorch Version: {torch.__version__}')
print(f'torchvision Version: {torchvision.__version__}')

PyTorch Version: 1.13.1
torchvision Version: 0.14.1


## 1. Data Preparation and Exploration

In [5]:
%load_ext tensorboard

In [6]:
%tensorboad --logdir logs

UsageError: Line magic function `%tensorboad` not found.
